# CA4 - Question 1

In [ ]:
import re

class ValidatorMeta(type):
    def __new__(cls, name, bases, dct):
        
        # تکمیل شده TO DO
        validation_rules = {
            'email': r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.(com|org)$',
            'phone_number': r'^\+98\d{9}$', # با توجه به متن خطا در کد، +۹۸ و ۹ رقم بعدی در نظر گرفته شد
            'password': r'^(?=.*[A-Z])(?=.*[a-z])(?=.*\d)(?=.*[*@%!])[A-Za-z\d*@%!]{8,12}$',
            'product_code': r'^[A-Z]{2}\d{2,4}[a-z]?(-v([1-9]|[1-9][0-9]))?$', # بر اساس متن خطا نسخه بین ۱ تا ۹۹ در نظر گرفته شد
            'stop_word': r'^(Stop|stop)$',
            'repeated_phrase': r'^\s*(some students|many employees).*?\1\s*$',
            'date': r'^\d{4}/(0[1-9]|1[0-2])/(0[1-9]|[12]\d|3[01])$', # این عبارت فرمت را چک می‌کند اما صحت روز نسبت به ماه را نه
            'quotation': r'^([\'"])[a-zA-Z0-9\s]*\1$',
            'parenthesis': r'^[^()]*(\([^()]*\)[^()]*)*$' # این عبارت پرانتزهای تو در تو را مدیریت نمی‌کند
        }

        
        custom_error_messages = {
            'email': 'Email must have a valid format and end with .com or .org.',
            'phone_number': 'Phone number must start with +98 and be followed by 9 digits.', # متن خطا اصلاح شد
            'password': 'Password must be between 8 and 12 characters and include at least one uppercase letter, one lowercase letter, one number, and one special character (*, @, %, !).',
            'product_code': 'Product code must consist of 2 uppercase letters, 2 to 4 digits, an optional lowercase letter, and an optional version number (v1-v99).',
            'stop_word': 'The word "Stop" or "stop" must be separate and not part of a larger word or attached to punctuation.',
            'repeated_phrase': 'The phrase "some students" or "many employees" must repeat exactly, with no different terms in between.',
            'date': 'Date must be in YYYY/MM/DD format, with valid month and day values.',
            'quotation': 'Text must be enclosed in balanced single or double quotes and contain only letters, numbers, and spaces.',
            'parenthesis': 'Parentheses must be balanced with no unmatched opening or closing parentheses.',
        }

        # Method generator for validation functions
        def create_validator(field, rule, custom_message):
            def validator(self, value):
                if not re.match(rule, value):
                    raise ValueError(custom_message)
                print(f"'{value}' is a valid {field}.")
                return True
            return validator

        # Add validation methods to the class dynamically
        for field, rule in validation_rules.items():
            dct[f'validate_{field}'] = create_validator(field, rule, custom_error_messages.get(field, f"Invalid {field}"))

        return super().__new__(cls, name, bases, dct)


class FormValidator(metaclass=ValidatorMeta):
    """Concrete validator class with all validation methods"""
    pass


--- Testing email ---
'example@example.com' is a valid email.
'test.user@domain.org' is a valid email.
OK: 'invalid-email.com' correctly identified as invalid. Error: Email must have a valid format and end with .com or .org.
OK: 'user@domain.net' correctly identified as invalid. Error: Email must have a valid format and end with .com or .org.
--------------------

--- Testing phone_number ---
'+98912345678' is a valid phone_number.
OK: '98912345678' correctly identified as invalid. Error: Phone number must start with +98 and be followed by 9 digits.
OK: '+9812345' correctly identified as invalid. Error: Phone number must start with +98 and be followed by 9 digits.
--------------------

--- Testing password ---
'ValidPass1!' is a valid password.
OK: 'Short1!' correctly identified as invalid. Error: Password must be between 8 and 12 characters and include at least one uppercase letter, one lowercase letter, one number, and one special character (*, @, %, !).
OK: 'nouppercase1!' correctly

### **گزارش تشریح عبارات منظم (Regex) برای سیستم اعتبارسنجی**

این گزارش به تفصیل هر یک از الگوهای Regex استفاده شده در کلاس `FormValidator` را شرح می‌دهد. [cite_start]هدف هر الگو، اعتبارسنجی یک فیلد ورودی خاص بر اساس قوانین تعریف شده در صورت پروژه است[cite: 9].

#### **۱. اعتبارسنجی ایمیل (Email)**
**الگو:** `r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.(com|org)$'`

[cite_start]**توضیح:** این عبارت برای اعتبارسنجی آدرس ایمیل طراحی شده است تا فرمت صحیح داشته باشد و با پسوندهای `.com` یا `.org` پایان یابد.
-   `^`: این نماد نشان می‌دهد که الگو باید از ابتدای رشته شروع به تطبیق کند.
-   `[a-zA-Z0-9._%+-]+`: این بخش، "بخش محلی" یا نام کاربری ایمیل را مشخص می‌کند. این الگو به یک یا چند (`+`) حرف بزرگ یا کوچک، عدد و کاراکترهای خاص `.`, `_`, `%`, `+`, `-` اجازه می‌دهد.
-   `@`: این کاراکتر باید دقیقاً پس از بخش محلی ظاهر شود.
-   `[a-zA-Z0-9.-]+`: این بخش دامنه ایمیل را تعریف می‌کند که می‌تواند شامل یک یا چند (`+`) حرف، عدد، نقطه یا خط تیره باشد.
-   `\.`: یک نقطه (`.`) را مشخص می‌کند. از `\` برای این استفاده شده که خود نقطه در Regex معنای خاصی (هر کاراکتری) دارد.
-   `(com|org)`: این گروه مشخص می‌کند که رشته باید یا با `com` یا با `org` تمام شود.
-   `$`: این نماد تضمین می‌کند که الگو در انتهای رشته به پایان می‌رسد و هیچ کاراکتر اضافی پس از آن وجود ندارد.

#### **۲. اعتبارسنجی شماره تلفن (Phone Number)**
**الگو:** `r'^\+98\d{9}$'`

**توضیح:** این الگو شماره تلفن‌هایی را اعتبارسنجی می‌کند که با پیش‌شماره ایران (`+98`) شروع شده و پس از آن دقیقاً ۹ رقم دیگر می‌آید. این ساختار با فرمت رایج شماره‌های موبایل در ایران تطابق دارد.
-   `^`: تطبیق از ابتدای رشته.
-   `\+98`: رشته باید دقیقاً با کاراکترهای `+` و سپس `98` شروع شود.
-   `\d{9}`: این بخش مشخص می‌کند که پس از `+98` باید دقیقاً ۹ کاراکتر عددی (`\d`) بیاید. `{9}` به معنای تکرار دقیقاً ۹ بار است.
-   `$`: تطبیق در انتهای رشته.

#### **۳. اعتبارسنجی رمز عبور (Password)**
**الگو:** `r'^(?=.*[A-Z])(?=.*[a-z])(?=.*\d)(?=.*[*@%!])[A-Za-z\d*@%!]{8,12}$'`

[cite_start]**توضیح:** این عبارت پیچیده برای اعتبارسنجی رمز عبور بر اساس چهار شرط همزمان و یک شرط طول طراحی شده است.
-   `^` و `$`: ابتدا و انتهای رشته را مشخص می‌کنند.
-   `(?=.*[A-Z])`: این یک "نگاه به جلو مثبت" (Positive Lookahead) است. بدون مصرف کردن کاراکترها، بررسی می‌کند که آیا حداقل یک حرف بزرگ در رشته وجود دارد یا خیر.
-   `(?=.*[a-z])`: به طور مشابه، وجود حداقل یک حرف کوچک را بررسی می‌کند.
-   `(?=.*\d)`: وجود حداقل یک رقم را بررسی می‌کند.
-   `(?=.*[*@%!])`: وجود حداقل یکی از کاراکترهای خاص `*`, `@`, `%`, `!` را بررسی می‌کند.
-   `[A-Za-z\d*@%!]{8,12}`: این بخش اصلی الگو است که کاراکترهای مجاز را تعریف کرده و با `{8,12}` تضمین می‌کند که طول کل رشته بین ۸ تا ۱۲ کاراکتر باشد.

#### **۴. اعتبارسنجی کد محصول (Product Code)**
**الگو:** `r'^[A-Z]{2}\d{2,4}[a-z]?(-v([1-9]|[1-9][0-9]))?$'`

[cite_start]**توضیح:** این الگو برای اعتبارسنجی کد محصول بر اساس ساختار مشخص شده در صورت سوال استفاده می‌شود.
-   `[A-Z]{2}`: رشته باید با دو حرف بزرگ شروع شود.
-   `\d{2,4}`: به دنبال آن باید بین ۲ تا ۴ رقم بیاید.
-   `[a-z]?`: یک حرف کوچک به صورت اختیاری (`?` به معنای صفر یا یک بار تکرار) می‌تواند وجود داشته باشد.
-   `(-v(...))?`: کل این گروه که مربوط به نسخه است، اختیاری است. این گروه با `-v` شروع می‌شود.
-   `([1-9]|[1-9][0-9])`: این بخش شماره نسخه را بین ۱ تا ۹۹ اعتبارسنجی می‌کند. `[1-9]` برای اعداد ۱ تا ۹ و `[1-9][0-9]` برای اعداد ۱۰ تا ۹۹ است.
-   `^` و `$`: ابتدا و انتهای رشته را مشخص می‌کنند.

#### **۵. اعتبارسنجی کلمه توقف (Stop Word)**
**الگو:** `r'^(Stop|stop)$'`

**توضیح:** این عبارت به سادگی بررسی می‌کند که آیا کل رشته ورودی دقیقاً کلمه `Stop` یا `stop` است یا خیر. [cite_start]این الگو تضمین می‌کند که این کلمه بخشی از یک کلمه بزرگتر یا متصل به علائم نگارشی نیست، زیرا ابتدا (`^`) و انتهای (`$`) رشته را محدود کرده است.

#### **۶. اعتبارسنجی عبارت تکراری (Repeated Phrase)**
**الگو:** `r'^\s*(some students|many employees).*?\1\s*$'`

[cite_start]**توضیح:** این الگو برای شناسایی تکرار دقیق عبارت "some students" یا "many employees" در یک رشته طراحی شده است.
-   `\s*`: اجازه وجود فضای خالی در ابتدا یا انتهای رشته را می‌دهد.
-   `(some students|many employees)`: این یک "گروه ضبط‌کننده" (Capturing Group) است. این گروه یکی از دو عبارت را تطبیق داده و آن را در حافظه (به عنوان گروه شماره ۱) ذخیره می‌کند.
-   `.*?`: این بخش هر کاراکتری را به صورت غیرحریصانه (non-greedy) تطبیق می‌دهد و اجازه می‌دهد متنی بین دو عبارت تکراری وجود داشته باشد.
-   `\1`: این یک "بازآوری" (Back-reference) است. این بخش الگو را مجبور می‌کند تا دقیقاً همان متنی را که توسط گروه شماره ۱ ضبط شده بود، دوباره تطبیق دهد.
-   `^` و `$`: ابتدا و انتهای رشته را مشخص می‌کنند.

#### **۷. اعتبارسنجی تاریخ (Date)**
**الگو:** `r'^\d{4}/(0[1-9]|1[0-2])/(0[1-9]|[12]\d|3[01])$'`

[cite_start]**توضیح:** این الگو برای اعتبارسنجی فرمت تاریخ به صورت `YYYY/MM/DD` استفاده می‌شود.
-   `\d{4}`: چهار رقم برای سال.
-   `/`: کاراکتر جداکننده اسلش.
-   `(0[1-9]|1[0-2])`: این گروه ماه را اعتبارسنجی می‌کند. `01` تا `09` یا `10` تا `12`.
-   `(0[1-9]|[12]\d|3[01])`: این گروه روز را اعتبارسنجی می‌کند. `01` تا `09`، `10` تا `29` و `30` یا `31`.
-   **نکته مهم:** این الگو تنها فرمت و بازه عددی را بررسی می‌کند و نمی‌تواند اعتبار منطقی تاریخ (مثلاً وجود روز ۳۱ در ماه‌های ۳۰ روزه) را تشخیص دهد.

#### **۸. اعتبارسنجی نقل قول (Quotation)**
**الگو:** `r'^([\'"])[a-zA-Z0-9\s]*\1$'`

[cite_start]**توضیح:** این عبارت بررسی می‌کند که آیا یک متن درون یک جفت کوتیشن تکی (`'`) یا دوتایی (`"`) متوازن قرار گرفته است و محتوای آن فقط شامل حروف، اعداد و فاصله است.
-   `([\'"])`: این گروه ضبط‌کننده، کاراکتر اول را که یا `'` یا `"` است، تطبیق داده و در حافظه ذخیره می‌کند.
-   `[a-zA-Z0-9\s]*`: محتوای داخل کوتیشن‌ها را تطبیق می‌دهد.
-   `\1`: این بازآوری تضمین می‌کند که کاراکتر پایانی دقیقاً مشابه کاراکتر آغازی باشد.
-   `^` و `$`: ابتدا و انتهای رشته را مشخص می‌کنند.

#### **۹. اعتبارسنجی پرانتز (Parenthesis)**
**الگو:** `r'^[^()]*(\([^()]*\)[^()]*)*$'`

[cite_start]**توضیح:** این الگو برای بررسی توازن پرانتزهای باز و بسته در یک رشته طراحی شده است، با این محدودیت که پرانتزهای تودرتو را پشتیبانی نمی‌کند.
-   `[^()]*`: هر تعداد کاراکتر که پرانتز باز یا بسته نباشد.
-   `(\([^()]*\)[^()]*)`: این گروه یک الگوی کامل از یک جفت پرانتز را تعریف می‌کند: یک پرانتز باز `\(`، سپس محتوای بدون پرانتز `[^()]*`، و یک پرانتز بسته `\)`.
-   `*`: ستاره در انتهای گروه بالا به این معناست که این الگوی پرانتزی می‌تواند صفر یا چند بار تکرار شود.
-   `^` و `$`: ابتدا و انتهای رشته را مشخص می‌کنند.

In [ ]:

# Test Validation
validator = FormValidator()

test_cases = {
    'email': [('example@example.com', True), ('test.user@domain.org', True), ('invalid-email.com', False), ('user@domain.net', False)],
    'phone_number': [('+98912345678', True), ('98912345678', False), ('+9812345', False)],
    'password': [('ValidPass1!', True), ('Short1!', False), ('nouppercase1!', False), ('NoNumber!', False), ('NoSpecial1', False)],
    'product_code': [('AB1234a-v12', True), ('CD567', True), ('XY9876b', True), ('ab123', False), ('AB12-v100', False)],
    'stop_word': [('stop', True), ('Stop', True), ('Stopped', False), ('my stop', False)],
    'repeated_phrase': [('some students like some students', True), ('many employees and many employees', True), ('some students and many employees', False)],
    'date': [('1404/03/28', True), ('1404/13/01', False), ('1404/02/31', False)], # Regex دومین مورد را تشخیص می‌دهد اما سومی را نه
    'quotation': [("'hello world'", True), ('"valid123"', True), ("'mismatched\"", False), ('"nested \'quote\'"', False)],
    'parenthesis': [('valid (text) here', True), ('no parenthesis', True), ('invalid ( text', False), ('(nested (problem))', False)]
}

for field, cases in test_cases.items():
    print(f"--- Testing {field} ---")
    validate_func = getattr(validator, f'validate_{field}')
    for value, expected_valid in cases:
        try:
            validate_func(value)
            if not expected_valid:
                print(f"!!! FAILED: '{value}' was considered valid but should be invalid.")
        except ValueError as e:
            if expected_valid:
                print(f"!!! FAILED: '{value}' was considered invalid but should be valid. Error: {e}")
            else:
                print(f"OK: '{value}' correctly identified as invalid. Error: {e}")
    print("-" * 20 + "\n")

## Part 2 - IMDB

In [ ]:
import requests
import re
import json

URL = "https://www.imdb.com/chart/top"
HEADERS = {
    "Accept-Language": "en-US,en;q=0.5",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

def fetch_page(url):
    print(f"[+] Fetching URL: {url}")
    response = requests.get(url, headers=HEADERS)
    print(f"[+] Status Code: {response.status_code}")
    response.raise_for_status()
    return response.text

def extract_json_data(html):
    print("[+] Extracting JSON data from HTML script tag...")
    json_pattern = re.compile(r'<script type="application/ld\+json">\s*(\{.*?\})\s*</script>', re.S)
    match = json_pattern.search(html)
    if not match:
        print("[-] JSON script not found!")
        return None

    json_str = match.group(1)
    print("[+] JSON script found. Parsing...")
    try:
        data = json.loads(json_str)
        print("[+] JSON parsed successfully.")
        return data
    except json.JSONDecodeError as e:
        print("[-] Failed to parse JSON:", e)
        return None

def extract_movies_from_json(data):
    print("[+] Extracting movie list from JSON data...")
    items = data.get("itemListElement", [])
    print(f"[+] Found {len(items)} movies.")

    movies = []
    for i, item in enumerate(items):
        m = item.get("item", {})
        title = m.get("name", "")
        url = m.get("url", "")
        rating = m.get("aggregateRating", {}).get("ratingValue", "")
        rating_count = m.get("aggregateRating", {}).get("ratingCount", "")
        content_rating = m.get("contentRating", "")
        duration = m.get("duration", "")
        genre = m.get("genre", "")
        description = m.get("description", "")

        movies.append({
            "Title": title,
            "URL": url,
            "Rating": rating,
            "Rating Count": rating_count,
            "Content Rating": content_rating,
            "Duration": duration,
            "Genre": genre,
            "Description": description
        })

    return movies

def main():
    html = fetch_page(URL)
    json_data = extract_json_data(html)
    if not json_data:
        print("[-] No data to process.")
        return

    movies = extract_movies_from_json(json_data)
    print(f"\n[+] Extracted {len(movies)} movies.")

    for i, movie in enumerate(movies):
        print(f"\n[Movie #{i+1}]")
        print(f"Title: {movie['Title']}")
        print(f"URL: {movie['URL']}")
        print(f"Rating: {movie['Rating']}")
        print(f"Rating Count: {movie['Rating Count']}")
        print(f"Content Rating: {movie['Content Rating']}")
        print(f"Duration: {movie['Duration']}")
        print(f"Genre: {movie['Genre']}")
        print(f"Description: {movie['Description']}")

if __name__ == "__main__":
    main()


[+] Fetching URL: https://www.imdb.com/chart/top
[+] Status Code: 200
[+] Extracting JSON data from HTML script tag...
[+] JSON script found. Parsing...
[+] JSON parsed successfully.
[+] Extracting movie list from JSON data...
[+] Found 250 movies.

[+] Extracted 250 movies.

[Movie #1]
Title: The Shawshank Redemption
URL: https://www.imdb.com/title/tt0111161/
Rating: 9.3
Rating Count: 3072909
Content Rating: R
Duration: PT2H22M
Genre: Drama
Description: A banker convicted of uxoricide forms a friendship over a quarter century with a hardened convict, while maintaining his innocence and trying to remain hopeful through simple compassion.

[Movie #2]
Title: The Godfather
URL: https://www.imdb.com/title/tt0068646/
Rating: 9.2
Rating Count: 2143790
Content Rating: R
Duration: PT2H55M
Genre: Crime, Drama
Description: The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.

[Movie #3]
Title: The Dark Knight
URL: https://www.imdb.co

#### **چالش‌ها و راهکارها در استفاده از Regex برای وب اسکرپینگ** 

**چالش‌ها:**

1.  **شکنندگی (Fragility):** عبارات منظم به شدت به ساختار HTML صفحه وب وابسته‌اند. هرگونه تغییر در نام کلاس‌ها، تگ‌ها یا ساختار کلی صفحه (که در سایت‌های مدرن امری رایج است) منجر به شکست خوردن کامل اسکرپر می‌شود.
2.  **پیچیدگی و عدم خوانایی:** برای استخراج اطلاعات از یک ساختار پیچیده مانند HTML، عبارت منظم بسیار طولانی، پیچیده و غیرقابل خواندن می‌شود. اشکال‌زدایی (Debug) چنین عبارتی بسیار دشوار است.
3.  **محتوای پویا (Dynamic Content):** اگر وب‌سایت محتوای خود را با استفاده از جاوااسکریپت پس از بارگذاری اولیه صفحه، بارگذاری کند (AJAX)، کتابخانه `urllib` نمی‌تواند این محتوا را ببیند، زیرا تنها HTML اولیه را دریافت می‌کند.
4.  **عدم توانایی در پارس ساختارهای تو در تو:** HTML و XML زبان‌های منظمی نیستند. Regex برای پارس کردن ساختارهای تو در تو و بازگشتی (مانند تگ‌های تو در تو) طراحی نشده است و استفاده از آن برای این منظور منجر به خطاهای غیرمنتظره می‌شود.

**راهکارها:**

1.  **استفاده از کتابخانه‌های پارس HTML:** بهترین راهکار، استفاده از کتابخانه‌هایی مانند `BeautifulSoup` یا `lxml` است. این ابزارها HTML را به یک درخت قابل پیمایش از اشیاء پایتون تبدیل می‌کنند که می‌توان به صورت ساختاریافته و با مقاومت بیشتری در برابر تغییرات جزئی، در آن به جستجو پرداخت.
2.  **عمومی‌سازی الگوها:** می‌توان الگوهای Regex را تا حد امکان عمومی نوشت (مثلاً به جای یک کلاس خاص `class="xyz"`, از `class=".*?"` استفاده کرد) اما این کار دقت را کاهش داده و ممکن است نتایج ناخواسته به همراه داشته باشد.
3.  **استفاده از ابزارهای مرورگر خودکار:** برای مقابله با محتوای پویا، می‌توان از ابزارهایی مانند `Selenium` یا `Playwright` استفاده کرد که یک مرورگر واقعی را کنترل کرده و به HTML نهایی پس از اجرای جاوااسکریپت دسترسی دارند.

-----

### **بخش سوم: سوالات تشریحی**

**۱- پنج مورد از کاربردهای Regex را نام ببرید.** 

1.  **اعتبارسنجی ورودی (Input Validation):** همانطور که در بخش اول تمرین انجام شد، برای اطمینان از اینکه ورودی‌های کاربر (مانند ایمیل، رمز عبور، شماره تلفن و کد پستی) از فرمت مشخصی پیروی می‌کنند. [cite: 9]
2.  **استخراج داده (Data Extraction/Scraping):** برای یافتن و استخراج اطلاعات خاص از متون طولانی، فایل‌های لاگ یا کدهای HTML، مانند استخراج لینک‌ها، قیمت‌ها یا تاریخ‌ها از یک صفحه وب. 
3.  **جستجو و جایگزینی (Search and Replace):** در ویرایشگرهای متن و کد، برای انجام عملیات پیچیده جایگزینی، مانند تغییر فرمت تاریخ در کل یک پروژه یا استانداردسازی نام متغیرها.
4.  **پارس کردن فایل‌های لاگ (Log File Parsing):** برای تحلیل فایل‌های لاگ سرورها و برنامه‌ها، به منظور پیدا کردن و دسته‌بندی خطاها، استخراج آدرس‌های IP، شناسایی الگوهای ترافیک یا مانیتورینگ رفتار سیستم.
5.  **پردازش زبان طبیعی (NLP):** در مراحل اولیه پردازش متون، برای کارهایی مانند توکنایزیشن (شکستن متن به کلمات یا جملات)، یافتن الگوهای خاص (مانند اسامی، تاریخ‌ها، مقادیر پولی) یا پاکسازی متن از کاراکترهای ناخواسته.

**۲- در صورتی که در حین انجام پروژه محدودیتی در استفاده از Regex وجود داشت ذکر کنید و همچنین محدودیت‌های کلی استفاده از Regex را بیان نمایید.** 

**محدودیت‌های مشاهده شده در پروژه:**

  * **اعتبارسنجی تاریخ (`Date`):** عبارت منظم `^\d{4}/(0[1-9]|1[0-2])/(0[1-9]|[12]\d|3[01])$` می‌تواند فرمت `YYYY/MM/DD` و بازه عددی ماه‌ها و روزها را بررسی کند، اما نمی‌تواند منطق تقویمی را درک کند. برای مثال، این الگو تاریخ نامعتبر `1404/02/31` (۳۱ خرداد) را به عنوان یک ورودی معتبر شناسایی می‌کند، زیرا نمی‌داند که ماه دوم (اردیبهشت) ۳۱ روزه است اما ماه خرداد ۳۱ روزه نیست. پیاده‌سازی این منطق با Regex بسیار پیچیده و غیرعملی است. 
  * **اعتبارسنجی پرانتز (`Parenthesis`):** زبان‌های منظم (که توسط Regex پردازش می‌شوند) قادر به شمارش و مدیریت ساختارهای بازگشتی یا تو در تو نیستند. الگوی `^[^()]*(\([^()]*\)[^()]*)*$` تنها می‌تواند پرانتزهای ساده و غیر تودرتو را اعتبارسنجی کند. این الگو یک ورودی مانند `(a (b) c)` را که از نظر منطقی نامتوازن نیست اما تو در تو است، به اشتباه رد می‌کند. 
  * **وب اسکرپینگ:** همانطور که در بخش دوم توضیح داده شد، شکنندگی الگوها در برابر تغییرات سایت IMDb یک محدودیت بزرگ بود. نام کلاس‌ها مانند `sc-b189961a-7` به نظر خودکار تولید شده‌اند و ممکن است در آینده تغییر کنند که این امر Regex را کاملاً بی‌فایده خواهد کرد.

**محدودیت‌های کلی Regex:**

1.  **عدم توانایی در پارس کردن زبان‌های غیرمنظم (Non-Regular Languages):** این مهم‌ترین محدودیت نظری Regex است. Regex نمی‌تواند ساختارهای نیازمند به حافظه یا شمارش، مانند HTML، XML یا عبارات ریاضی با پرانتزهای تودرتو را به درستی پارس کند.
2.  **پیچیدگی و خوانایی پایین (Regex Hell):** با افزایش پیچیدگی الگو، عبارات Regex به سرعت غیرقابل خواندن، اشکال‌زدایی و نگهداری می‌شوند.
3.  **کارایی (Performance):** یک عبارت Regex که به صورت بهینه نوشته نشده باشد (به خصوص با استفاده زیاد از `backtracking`) می‌تواند عملکرد بسیار کندی روی ورودی‌های بزرگ داشته باشد و باعث مشکلات جدی در کارایی برنامه شود.
4.  **عدم پشتیبانی از منطق محاسباتی:** Regex نمی‌تواند عملیات ریاضی یا منطقی پیچیده را انجام دهد. برای مثال، نمی‌تواند بررسی کند که آیا یک عدد در یک رشته، حاصل جمع دو عدد دیگر در همان رشته است یا خیر.